In [10]:
import pandas as pd

# 데이터셋 URL
train_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt"
test_url = "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"

train_df = pd.read_csv(train_url, sep='\t')
test_df = pd.read_csv(test_url, sep='\t')

# 데이터 상위 5개 확인
display(train_df.head())
display(test_df.head())

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [11]:
train_df.dropna(how='any', inplace=True)
test_df.dropna(how='any', inplace=True)

In [12]:
# 중복된 행 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

print("중복 제거 후 train_df 크기:", len(train_df))
print("중복 제거 후 test_df 크기:", len(test_df))

중복 제거 후 train_df 크기: 146182
중복 제거 후 test_df 크기: 49157


다음 단계는 텍스트 데이터에 대한 토큰화 및 어휘집을 생성하는 것입니다. 한국어 텍스트 처리를 위해 `konlpy` 라이브러리의 Okt 형태소 분석기를 사용할 수 있습니다.

In [13]:
import re
from konlpy.tag import Okt

okt = Okt()

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = [line.strip() for line in f]
    return stopwords

ko_stopwords = load_stopwords('ko_stopwords.txt')

def preprocess(text, return_tokens=False):
    text = re.sub("[^가-힣 ]", "", text)
    tokens = okt.morphs(text, stem=True)
    tokens = [word for word in tokens if word not in ko_stopwords]
    return tokens if return_tokens else ' '.join(tokens)

In [14]:
# preprocess 함수를 이용하여 document 열 전처리
train_df['processed_document'] = train_df['document'].apply(lambda x: preprocess(x))
test_df['processed_document'] = test_df['document'].apply(lambda x: preprocess(x))

# 결과 확인
display(train_df.head())
display(test_df.head())

,id,document,label,processed_document
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,더빙 진짜 짜증나다 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠 포스터 보고 초딩 영화 줄 오버 연기 가볍다 않다
2,10265843,너무재밓었다그래서보는것을추천한다,0,무재 밓었 다그 래서 보다 추천 한 다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기 구먼 솔직하다 재미 는 없다 평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이 몬페 익살스럽다 연기 돋보이다 영화 스파이더맨 늙다 보이다 하다 커스틴 던스트...


,id,document,label,processed_document
0,6270596,굳 ㅋ,1,굳다
1,9274899,GDNTOPCLASSINTHECLUB,0,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐 평점 은 나쁘다 않다 점 짜다 리 는 더 더욱 아니다
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하다 않다 완전 막장 임 돈 주다 보기 에는
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니다 별 개 주다 나오다 심기 불편하다 하다


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# TF-IDF 벡터화
# train_df와 test_df의 'processed_document' 열을 사용하여 벡터화합니다.
# 모델 학습 시 훈련 데이터에 나타난 단어만 사용해야 하므로 train_df를 fit_transform에 사용합니다.
tfidf_vectorizer = TfidfVectorizer(max_features=3000) # 예시로 상위 3000개의 특성만 사용
X_train = tfidf_vectorizer.fit_transform(train_df['processed_document'])
X_test = tfidf_vectorizer.transform(test_df['processed_document'])

y_train = train_df['label']
y_test = test_df['label']


print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (146182, 3000)
X_test shape: (49157, 3000)


In [17]:
# 어휘집 생성 및 시퀀스를 정수 인덱스로 변환
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df['processed_document'])

# 텍스트 시퀀스를 정수 시퀀스로 변환
train_sequences_indexed = tokenizer.texts_to_sequences(train_df['processed_document'])
test_sequences_indexed = tokenizer.texts_to_sequences(test_df['processed_document'])

# 시퀀스 패딩 (정수 시퀀스에 대해 수행)
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(
    train_sequences_indexed,
    maxlen=max_sequence_length,
    padding='post',
    truncating='post'
)
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(
    test_sequences_indexed,
    maxlen=max_sequence_length,
    padding='post',
    truncating='post'
)

print("X_train_padded shape:", X_train_padded.shape)
print("X_test_padded shape:", X_test_padded.shape)

X_train_padded shape: (146182, 31)
X_test_padded shape: (49157, 31)


In [18]:
import tensorflow as tf

# 임베딩 레이어 설정
# vocab_size는 tokenizer의 word_index 크기 + 1 (0번 인덱스를 패딩에 사용하므로)
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100  # 원하는 임베딩 벡터 크기
input_length = max_sequence_length # 패딩된 시퀀스 길이

# 임베딩 레이어 생성
embedding_layer = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    input_length=input_length
)

# 임베딩 레이어 구성 확인
print("Embedding Layer Configuration:")
print(embedding_layer.get_config())

Embedding Layer Configuration:
{'name': 'embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'input_dim': 42831, 'output_dim': 100, 'embeddings_initializer': {'module': 'keras.initializers', 'class_name': 'RandomUniform', 'config': {'seed': None, 'minval': -0.05, 'maxval': 0.05}, 'registered_name': None}, 'embeddings_regularizer': None, 'activity_regularizer': None, 'embeddings_constraint': None, 'mask_zero': False}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# LSTM 모델 정의
model = Sequential()
model.add(embedding_layer) # 이전에 준비한 임베딩 레이어 추가
model.add(LSTM(units=64)) # LSTM 레이어 추가 (units는 예시값)
model.add(Dense(units=32, activation='relu')) # 추가적인 Dense 레이어 (선택사항)
model.add(Dense(units=1, activation='sigmoid')) # 이진 분류를 위한 최종 Dense 레이어

# 모델 요약 출력
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [20]:
# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습
history = model.fit(
    X_train_padded,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(X_test_padded, y_test)
)

Epoch 1/5
2285/2285 ━━━━━━━━━━━━━━━━━━━━ 208s 89ms/step - accuracy: 0.6516 - loss: 0.5723 - val_accuracy: 0.8272 - val_loss: 0.3759
Epoch 2/5
2285/2285 ━━━━━━━━━━━━━━━━━━━━ 267s 92ms/step - accuracy: 0.8400 - loss: 0.3520 - val_accuracy: 0.8355 - val_loss: 0.3592
Epoch 3/5
2285/2285 ━━━━━━━━━━━━━━━━━━━━ 251s 87ms/step - accuracy: 0.8539 - loss: 0.3249 - val_accuracy: 0.8359 - val_loss: 0.3671
Epoch 4/5
2285/2285 ━━━━━━━━━━━━━━━━━━━━ 201s 88ms/step - accuracy: 0.8622 - loss: 0.3057 - val_accuracy: 0.8371 - val_loss: 0.3625
Epoch 5/5
2285/2285 ━━━━━━━━━━━━━━━━━━━━ 211s 92ms/step - accuracy: 0.8722 - loss: 0.2870 - val_accuracy: 0.8348 - val_loss: 0.3681


In [28]:
# 모델 저장
model.save('my_sentiment_model.keras')

# 모델 로드
model = tf.keras.models.load_model('my_sentiment_model.keras')

In [29]:
import numpy as np
import tensorflow as tf

# 예측 함수 정의
def predict_sentiment(text, tokenizer, model, max_sequence_length):
    processed_text = preprocess(text)

    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen=max_sequence_length,
        padding='post',
        truncating='post'
    )
    prediction = model.predict(padded_sequence, verbose=0)
    sentiment_score = prediction[0][0]

    sentiment = "긍정 (Positive)" if sentiment_score > 0.5 else "부정 (Negative)"

    return sentiment, sentiment_score

In [30]:
input_text = input("감성 분석할 텍스트를 입력하세요: ")

# 정의된 예측 함수를 사용하여 감성 분석 수행
predicted_sentiment, sentiment_score = predict_sentiment(
    input_text,
    tokenizer,
    model,
    max_sequence_length
)

print(f"\n입력 텍스트: '{input_text}'")
print(f"예측 값 (확률): {sentiment_score:.4f}")
print(f"예측 감성: {predicted_sentiment}")

감성 분석할 텍스트를 입력하세요: 발연기

입력 텍스트: '발연기'
예측 값 (확률): 0.0082
예측 감성: 부정 (Negative)
